In [1]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns

In [9]:
df = pd.read_csv(r'../data/game_details.csv')
df.head()

,game_name,highest_price,release_date,genre,publisher,platform,metacritic_score,metacritic_rating_count,metacritic_user_score,metacritic_user_rating_count,playstation_score,playstation_rating_count
0,Grand Theft Auto IV,€24.99,"Feb 15, 2012",Action / Shooter / Racing,Rockstar,PS3,98.0,86.0,8.3,5541.0,4.32,48904.0
1,Red Dead Redemption 2,€59.99,"Oct 26, 2018",Action / Adventure / Unique,Rockstar Games,PS4,97.0,99.0,8.9,31932.0,4.74,379257.0
2,Red Dead Online,€69.99,"Oct 29, 2018",Action / Adventure,Rockstar Games,PS4,97.0,99.0,8.9,31932.0,4.74,379346.0
3,Grand Theft Auto 3,€9.99,"Oct 4, 2012",--,Rockstar Games,PS3,97.0,56.0,8.0,2079.0,4.59,1437.0
4,Grand Theft Auto V,€69.99,"Sep 17, 2013",Action / Adventure,Rockstar Games,PS3,97.0,66.0,8.5,14322.0,4.65,40895.0


In [8]:
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 3526 entries, 0 to 3525
Data columns (total 12 columns):
 #   Column                        Non-Null Count  Dtype  
---  ------                        --------------  -----  
 0   game_name                     3526 non-null   object 
 1   highest_price                 3415 non-null   object 
 2   release_date                  3526 non-null   object 
 3   genre                         3526 non-null   object 
 4   publisher                     3471 non-null   object 
 5   platform                      3526 non-null   object 
 6   metacritic_score              879 non-null    float64
 7   metacritic_rating_count       879 non-null    float64
 8   metacritic_user_score         880 non-null    float64
 9   metacritic_user_rating_count  880 non-null    float64
 10  playstation_score             3526 non-null   object 
 11  playstation_rating_count      2557 non-null   float64
dtypes: float64(5), object(7)
memory usage: 330.7+ KB


In [24]:
df.shape

(3526, 12)

Após analisar a estrutura do dataframe, listo abaixo alguns ajustes necessários para que as análises entreguem os melhores resultados:

Tratamento das colunas:

1. na coluna highest_price, tirar o símbolo do euro e transformar em float
2. transformar a coluna release_date para datetime
3. na coluna genre, dividir os que tem mais de um genero, tendo uma coluna de genero principal
4. tranformar a coluna playstation_score para numérica
5. transformar as colunas metacritic_rating_count e playstation_rating_count para int
Valores Nulos: 
6. Para Análise Exploratória: Você pode preencher com o 0 para indicar que não há avaliações.
7. Para Machine Learning: É mais comum preencher com a média ou mediana da coluna, ou criar uma coluna dummy (boolean) para indicar se o valor estava ausente.
Standartização de texto:
8. Converta todas as strings (genre, publisher, platform) para minúsculas (.str.lower()) e remova espaços em excesso no início e no fim (.str.strip()).

In [22]:
df['highest_price'].unique()

array(['€24.99', '€59.99', '€69.99', '€9.99', nan, '€14.99', '€49.99',
       '€19.99', '€29.99', '€17.99', '€39.99', '€79.99', '€5.99', '€4.99',
       '€7.99', '€12.99', '€54.99', '€8.99', '€13.99', '€1.99', '€18.99',
       '€12.49', '€21.99', '€23.99', '€14.49', '€11.99', '€34.99',
       '€2.99', '€100.00', '€36.99', '€10,000.00', '€74.99', 'FREE',
       '€6.99', '€26.99', '€44.99', '€3.99', '€16.99', '€28.99', '€11.25',
       '€28.49', '€15.99', '€10.99', '€13.29', '€20.99', '€99.99',
       '€5.49', '€32.99', '€38.99', '€7.49', '€64.99', '€2.49', '€6.19',
       '€35.99', '€4.49', '€70.00', '€3.59', '€13.49', '€6.49', '€31.99',
       '€37.99', '€6.59', '€1.49'], dtype=object)

1 - Para ser possível alterar o tipo da coluna 'highest_price' para float, será necessário excluir as linhas com valores como '€10,000.00' (pois impedem a alteração do tipo distorcem as análises gerais) e substituir o 'FREE' por 0.

In [29]:
# Filtra o DataFrame, mantendo apenas as linhas onde o preço é diferente do valor problemático

df = df[df['highest_price'] != '€10,000.00']

# Substitui os valores que estão como 'FREE' por 0

df['highest_price'] = df['highest_price'].replace('FREE', 0)

In [30]:
df['highest_price'].unique()

array(['€24.99', '€59.99', '€69.99', '€9.99', nan, '€14.99', '€49.99',
       '€19.99', '€29.99', '€17.99', '€39.99', '€79.99', '€5.99', '€4.99',
       '€7.99', '€12.99', '€54.99', '€8.99', '€13.99', '€1.99', '€18.99',
       '€12.49', '€21.99', '€23.99', '€14.49', '€11.99', '€34.99',
       '€2.99', '€100.00', '€36.99', '€74.99', 0, '€6.99', '€26.99',
       '€44.99', '€3.99', '€16.99', '€28.99', '€11.25', '€28.49',
       '€15.99', '€10.99', '€13.29', '€20.99', '€99.99', '€5.49',
       '€32.99', '€38.99', '€7.49', '€64.99', '€2.49', '€6.19', '€35.99',
       '€4.49', '€70.00', '€3.59', '€13.49', '€6.49', '€31.99', '€37.99',
       '€6.59', '€1.49'], dtype=object)

In [32]:
# Antes de alterar o tipo da coluna para numérico (float), é necessário retirar o símbolo '€' e também garantir que não haverá espaços antes ou depois dos valores (usando o strip())

df['highest_price'] = (df['highest_price'].str.replace('€','').str.strip().astype(float))

In [34]:
df.info()

<class 'pandas.core.frame.DataFrame'>
Index: 3515 entries, 0 to 3525
Data columns (total 12 columns):
 #   Column                        Non-Null Count  Dtype  
---  ------                        --------------  -----  
 0   game_name                     3515 non-null   object 
 1   highest_price                 3373 non-null   float64
 2   release_date                  3515 non-null   object 
 3   genre                         3515 non-null   object 
 4   publisher                     3460 non-null   object 
 5   platform                      3515 non-null   object 
 6   metacritic_score              877 non-null    float64
 7   metacritic_rating_count       877 non-null    float64
 8   metacritic_user_score         878 non-null    float64
 9   metacritic_user_rating_count  878 non-null    float64
 10  playstation_score             3515 non-null   object 
 11  playstation_rating_count      2546 non-null   float64
dtypes: float64(6), object(6)
memory usage: 357.0+ KB


2. transformar a coluna release_date para datetime

In [40]:
df.head(5)

,game_name,highest_price,release_date,genre,publisher,platform,metacritic_score,metacritic_rating_count,metacritic_user_score,metacritic_user_rating_count,playstation_score,playstation_rating_count
0,Grand Theft Auto IV,24.99,2012-02-15,Action / Shooter / Racing,Rockstar,PS3,98.0,86.0,8.3,5541.0,4.32,48904.0
1,Red Dead Redemption 2,59.99,2018-10-26,Action / Adventure / Unique,Rockstar Games,PS4,97.0,99.0,8.9,31932.0,4.74,379257.0
2,Red Dead Online,69.99,2018-10-29,Action / Adventure,Rockstar Games,PS4,97.0,99.0,8.9,31932.0,4.74,379346.0
3,Grand Theft Auto 3,9.99,2012-10-04,--,Rockstar Games,PS3,97.0,56.0,8.0,2079.0,4.59,1437.0
4,Grand Theft Auto V,69.99,2013-09-17,Action / Adventure,Rockstar Games,PS3,97.0,66.0,8.5,14322.0,4.65,40895.0


In [36]:
df['release_date'] = pd.to_datetime(df['release_date'], errors='coerce')

In [38]:
df.info()

<class 'pandas.core.frame.DataFrame'>
Index: 3515 entries, 0 to 3525
Data columns (total 12 columns):
 #   Column                        Non-Null Count  Dtype         
---  ------                        --------------  -----         
 0   game_name                     3515 non-null   object        
 1   highest_price                 3373 non-null   float64       
 2   release_date                  3515 non-null   datetime64[ns]
 3   genre                         3515 non-null   object        
 4   publisher                     3460 non-null   object        
 5   platform                      3515 non-null   object        
 6   metacritic_score              877 non-null    float64       
 7   metacritic_rating_count       877 non-null    float64       
 8   metacritic_user_score         878 non-null    float64       
 9   metacritic_user_rating_count  878 non-null    float64       
 10  playstation_score             3515 non-null   object        
 11  playstation_rating_count      2546 

In [41]:
df.head(5)

,game_name,highest_price,release_date,genre,publisher,platform,metacritic_score,metacritic_rating_count,metacritic_user_score,metacritic_user_rating_count,playstation_score,playstation_rating_count
0,Grand Theft Auto IV,24.99,2012-02-15,Action / Shooter / Racing,Rockstar,PS3,98.0,86.0,8.3,5541.0,4.32,48904.0
1,Red Dead Redemption 2,59.99,2018-10-26,Action / Adventure / Unique,Rockstar Games,PS4,97.0,99.0,8.9,31932.0,4.74,379257.0
2,Red Dead Online,69.99,2018-10-29,Action / Adventure,Rockstar Games,PS4,97.0,99.0,8.9,31932.0,4.74,379346.0
3,Grand Theft Auto 3,9.99,2012-10-04,--,Rockstar Games,PS3,97.0,56.0,8.0,2079.0,4.59,1437.0
4,Grand Theft Auto V,69.99,2013-09-17,Action / Adventure,Rockstar Games,PS3,97.0,66.0,8.5,14322.0,4.65,40895.0
